In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import zipfile
from helper import spkly

In [2]:
zip = zipfile.ZipFile('../data/Teleost_Especes_Pelagiques_Abondance_2009-2018.zip')
zip.namelist()

# let's get the fish mappings - requires specific encoding to be read
FISH_MAP = pd.read_csv('../data/DataDictionary_DictionnaireDonnees.csv', encoding='cp437', skiprows=10).set_index('Species')
FISH_MAP = FISH_MAP[['Name _EN']].to_dict('dict')['Name _EN']
FISH_MAP

{'AMSPP': 'Sand Lances (Ammodytes sp.)',
 'ARRIS': 'White Barracudina',
 'ARSIL': 'Atlantic Argentine',
 'BOSAI': 'Arctic Cod',
 'CLHAR': 'Atlantic Herring',
 'CYLUM': 'Lumpfish',
 'GAACU': 'Threespine Stickleback',
 'MAVIL': 'Capelin (M. villosus)',
 'MEATL': 'Atlantic Soft Pout',
 'MEBIL': 'Silver Hake',
 'OSMOR': 'Rainbow Smelt',
 'POVIR': 'Pollock',
 'SCSCO': 'Atlantic Mackerel (S. scombrus)'}

In [3]:
import datetime 

north_fish, south_fish = pd.read_csv(zip.open(zip.namelist()[0])), pd.read_csv(zip.open(zip.namelist()[1]))
north_fish['region'] = 'NORTH'
south_fish['region'] = 'SOUTH'

# let's just append the two datasets together and rename the fish mappings to something slightly more human readable 
fish_set = pd.concat([north_fish, south_fish]).rename(columns=FISH_MAP).copy()
fish_set['Date'] = pd.to_datetime(fish_set['Date'])
display(fish_set.shape)

# these sparkline image volumes don't look very promising for some species... e.g., sand lances. Also see that this data is only collected 3
# months out of a year!
spkly.display(fish_set.groupby([fish_set['Date'].dt.year, fish_set['Date'].dt.month]).sum())

(3165, 19)

In [4]:
fish_set.dtypes

Date                               datetime64[ns]
Station                                     int64
Longitude                                 float64
Latitude                                  float64
Profondeur/Depth                          float64
Sand Lances (Ammodytes sp.)               float64
White Barracudina                         float64
Atlantic Argentine                        float64
Arctic Cod                                float64
Atlantic Herring                          float64
Lumpfish                                  float64
Threespine Stickleback                    float64
Capelin (M. villosus)                     float64
Atlantic Soft Pout                        float64
Silver Hake                               float64
Rainbow Smelt                             float64
Pollock                                   float64
Atlantic Mackerel (S. scombrus)           float64
region                                     object
dtype: object

In [5]:
import folium

import folium
m = folium.Map(location=[50, -60], zoom_start=4.5, tiles='Stamen Terrain')

# testing on the first 10 points.
for flt in fish_set.index[:10]:
    folium.Marker(location=[fish_set["Latitude"].iloc[flt], fish_set["Longitude"].iloc[flt]]).add_to(m)
m

In [6]:
# pushing it to database

# for inserting on dataframes 

import zipfile
import pandas as pd
zip = zipfile.ZipFile('../data/Teleost_Especes_Pelagiques_Abondance_2009-2018.zip')
zip.namelist()

# let's get the fish mappings - requires specific encoding to be read
FISH_MAP = pd.read_csv('../data/DataDictionary_DictionnaireDonnees.csv', encoding='cp437', skiprows=10).set_index('Species')
FISH_MAP = FISH_MAP[['Name _EN']].to_dict('dict')['Name _EN']
north_fish, south_fish = pd.read_csv(zip.open(zip.namelist()[0])), pd.read_csv(zip.open(zip.namelist()[1]))
north_fish['region'] = 'NORTH'
south_fish['region'] = 'SOUTH'

# let's just append the two datasets together and rename the fish mappings to something slightly more human readable 
fish_set = pd.concat([north_fish, south_fish]).rename(columns=FISH_MAP).copy()
fish_set['Date'] = pd.to_datetime(fish_set['Date'])
fish_set.columns = [i.lower() for i in fish_set.columns]
DATABASE_COLUMN_MAPPING = {'profondeur/depth': 'depth', 'sand lances (ammodytes sp.)': 'sand_lances', 'white barracudina': 'white_barracudina', 
                          'atlantic argentine': 'atlantic_argentine', 'arctic cod': 'arctic_cod', 'atlantic herring': 'atlantic_herring', 
                          'threespine stickleback': 'threespine_stickleback', 'capelin (m. villosus)': 'capelin', 'atlantic soft pout': 'atlantic_soft_pout',
                          'silver hake': 'silver_hake', 'rainbow smelt': 'rainbow_smelt', 'atlantic mackerel (s. scombrus)': 'atlantic_mackerel'}
fish_set.rename(columns=DATABASE_COLUMN_MAPPING, inplace=True)

melted_columns = [i for i in fish_set.columns if i not in ['date', 'station', 'longitude', 'latitude', 'depth', 'region']]
melted_fish_set = pd.melt(fish_set, id_vars=['date', 'station', 'longitude', 'latitude', 'depth', 'region'], value_vars=melted_columns)
melted_fish_set.rename(columns={'value': 'fish_count', 'variable': 'fish_type'}, inplace=True)
display(melted_fish_set)
# db.insert_table(table_name='fish_data', df=melted_fish_set)

,date,station,longitude,latitude,depth,region,fish_type,fish_count
0,2009-08-01,8,-59.652417,47.724500,435.0,NORTH,sand_lances,0.0
1,2009-08-01,9,-59.851500,47.833167,502.0,NORTH,sand_lances,0.0
2,2009-08-01,10,-60.046167,47.752250,513.5,NORTH,sand_lances,0.0
3,2009-08-01,11,-60.635833,47.847167,308.5,NORTH,sand_lances,0.0
4,2009-08-02,12,-60.795750,47.998917,383.5,NORTH,sand_lances,0.0
...,...,...,...,...,...,...,...,...
41140,2018-10-01,105,-61.127333,47.353500,48.0,SOUTH,atlantic_mackerel,3.0
41141,2018-10-01,106,-61.023667,47.513167,41.0,SOUTH,atlantic_mackerel,0.0
41142,2018-10-01,107,-60.665000,47.427167,62.0,SOUTH,atlantic_mackerel,0.0
41143,2018-10-01,108,-60.641833,47.230500,106.0,SOUTH,atlantic_mackerel,0.0
